<a href="https://colab.research.google.com/github/saransh121/GEN-AI/blob/main/AutoSuggetion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00


## load model

In [18]:
import torch
from  torch import nn
from torch.utils.data import DataLoader
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
print(device)

cuda


In [19]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.unk_token})
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## Test use case

In [ ]:
data = tokenizer('SELECT creation ',max_length=10,return_tensors='pt',truncation=True)

In [ ]:
data.to(device)

{'input_ids': tensor([[46506,  6282,   220]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}

In [ ]:
pred = model.generate(**data,pad_token_id=50258)
tokenizer.decode(pred[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'SELECT creation ids = new ArrayList<String>();\n\n\n\n\n\n\n\n'

In [ ]:
pred.shape

torch.Size([1, 20])

#loading the dataset
 ## convert the string in to sql string by llm


In [4]:
from datasets import load_dataset
dataset = load_dataset('spider')

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

### data visualization

In [ ]:
dataset['train'][8]

{'db_id': 'department_management',
 'query': 'SELECT creation FROM department GROUP BY creation ORDER BY count(*) DESC LIMIT 1',
 'question': 'In which year were most departments established?',
 'query_toks': ['SELECT',
  'creation',
  'FROM',
  'department',
  'GROUP',
  'BY',
  'creation',
  'ORDER',
  'BY',
  'count',
  '(',
  '*',
  ')',
  'DESC',
  'LIMIT',
  '1'],
 'query_toks_no_value': ['select',
  'creation',
  'from',
  'department',
  'group',
  'by',
  'creation',
  'order',
  'by',
  'count',
  '(',
  '*',
  ')',
  'desc',
  'limit',
  'value'],
 'question_toks': ['In',
  'which',
  'year',
  'were',
  'most',
  'departments',
  'established',
  '?']}

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [7]:
pd.DataFrame(dataset['train']).head()

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...","[How, many, heads, of, the, departments, are, ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","[List, the, name, ,, born, state, and, age, of..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","[List, the, creation, year, ,, name, and, budg..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,"[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...","[What, are, the, maximum, and, minimum, budget..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...","[What, is, the, average, number, of, employees..."


In [ ]:
## ref : https://github.com/huggingface/notebooks/blob/main/examples/summarization.ipynb

In [9]:
def get_predication(text):
  data = tokenizer(text=text,max_length=10,return_tensors='pt',truncation=True)
  data.to(device)
  pred = model.generate(**data)
  return tokenizer.decode(pred[0])

### get the tokenized data

In [ ]:
max_input_length = 10
max_target_length = 10
def get_toknizer(text):
  model_inputs =  tokenizer(text = text['question'],return_tensors='pt',max_length=max_input_length,padding='max_length',truncation=True)
  lables = tokenizer(text_target = text['query'],return_tensors='pt',max_length=max_target_length,padding='max_length', truncation=True)
  return model_inputs,lables['input_ids']

In [11]:
training_data = dataset['train'].select(range(1000))
test_data = dataset['validation'].select(range(147))

In [ ]:
_ , _ = get_toknizer(training_data[0])

In [ ]:
tokenizer.pad_token_id

50256

### data loader class define

In [ ]:
class suggetionDataSet():
  def __init__(self,data,get_toknizer=get_toknizer):
    super().__init__();
    self.data = data
    self.get_toknizer = get_toknizer
  def __len__(self):
    return len(self.data)
  def __getitem__(self,index):
    model_inputs_data , labels_data = self.get_toknizer(self.data[index])
    return model_inputs_data,labels_data

In [ ]:
model_input_dataSet = suggetionDataSet(training_data)

In [ ]:
model_input_dataSet[8]

({'input_ids': tensor([[  818,   543,   614,   547,   749, 13346,  4920,    30, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])},
 tensor([[46506,  6282, 16034,  5011, 44441, 11050,  6282, 38678, 11050,   954]]))

In [ ]:
model_input_dataSet[0][0]['input_ids']

tensor([[ 2437,   867,  6665,   286,   262, 13346,   389,  4697,   621,  7265]])

In [ ]:
model_input_dataSet[0][1].shape

torch.Size([1, 10])

In [ ]:
model_input_dataSet[8][0]['input_ids']

tensor([[  818,   543,   614,   547,   749, 13346,  4920,    30, 50256, 50256]])

In [ ]:
model_input_dataSet[7][0]['input_ids']

tensor([[2061,  389,  262, 3891,  286,  262, 2585,  810,  379, 1551]])

### predication before training

In [20]:
get_predication("select data")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'select data.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

### creating batches

In [ ]:
for batch, labels in model_input_dataSet:
        batch = batch.to(device)
        print(batch['input_ids'].shape[1])

        break

10


### finetune

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
epoches = 50

In [ ]:
from tqdm.auto import tqdm

In [ ]:
model.train()
torch.cuda.empty_cache()
count = 0
for epoch in tqdm(range(epoches)):
    for batch, labels in model_input_dataSet:
        batch = batch.to(device)
        input_ids_shape = batch['input_ids'].shape[1]
        labels = labels[:,:input_ids_shape]
        # labels.replace(50257,0)
        labels = labels.to(device)
        optimizer.zero_grad()
        # Forward pass
        # print(f"batch['input_ids'] : {batch['input_ids']} : shape {batch['input_ids'].shape}")
        # print(f"batch['attention_mask'] : {batch['attention_mask']} : shape : {batch['attention_mask'].shape}")
        # print(f"labels  {labels} shape :: {labels.shape}")

        output = model(**batch, labels=labels)
        loss = output.loss
        # print(f"loss :  {loss}")

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Print or log the loss after each epoch
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

    # Optional: Clear GPU cache to prevent memory overflow
    torch.cuda.empty_cache()

# Set the model back to evaluation mode after training
model.eval()

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1, Loss: 5.401601791381836
Epoch 2, Loss: 4.347717761993408
Epoch 3, Loss: 3.5797712802886963
Epoch 4, Loss: 3.4419631958007812
Epoch 5, Loss: 3.157705307006836
Epoch 6, Loss: 3.219980478286743
Epoch 7, Loss: 2.6226797103881836
Epoch 8, Loss: 2.6309800148010254
Epoch 9, Loss: 2.68914794921875
Epoch 10, Loss: 1.9940539598464966
Epoch 11, Loss: 2.36617374420166
Epoch 12, Loss: 1.88271963596344
Epoch 13, Loss: 3.0005273818969727
Epoch 14, Loss: 2.0017852783203125
Epoch 15, Loss: 1.890918493270874
Epoch 16, Loss: 1.6501554250717163
Epoch 17, Loss: 1.952068567276001
Epoch 18, Loss: 2.13643741607666
Epoch 19, Loss: 1.9661481380462646
Epoch 20, Loss: 1.290059208869934
Epoch 21, Loss: 2.0365283489227295
Epoch 22, Loss: 1.7264169454574585
Epoch 23, Loss: 1.190832257270813
Epoch 24, Loss: 1.5280113220214844
Epoch 25, Loss: 1.4214110374450684
Epoch 26, Loss: 1.4882631301879883
Epoch 27, Loss: 1.528082013130188
Epoch 28, Loss: 1.3205938339233398
Epoch 29, Loss: 2.1397247314453125
Epoch 30, L

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## saving the models and checkpoints

In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/Colab Notebooks/tokonizedText_Autosuggest/model_checkPoint')

In [ ]:
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/tokonizedText_Autosuggest/model')

### loading the model

In [21]:
model = torch.load('/content/drive/MyDrive/Colab Notebooks/tokonizedText_Autosuggest/model')
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/tokonizedText_Autosuggest/model_checkPoint'))
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

### predication after fine tune

In [22]:
get_predication("select data")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'select data2.Name_code FROM FROM WHERE EX WHERE EX WHERE EX WHERE EX WHERE EX WHERE'